In [1]:
import  pandas as pd
import ast
from bs4 import BeautifulSoup
import requests
import numpy as np

PATH_IN = './datasets/moviedata/'

First, we start from the dataset obtained in the file named projectMilestone2.ipynb with a few modifications/ discussions we find necessary after more in-depth analysis performed on the dataset:
1. We decided to keep movies longer than 300 minutes, since after closer verification they turn out to be valid movies (except the ones that will be corrected in the following cells).
2. In the feedback for Milestone 2 it was mentioned that we don't do anything with movies that last 0 minutes. We checked them on IMDb website, and they are valid movies. Movie industry, in its infancy produced extremely short from current perspective 'movies'. Even though they are short, they exist as examples of the beginning of cinematography.
3. It was also pointed out that one of release years is 1010. Since this is clearly a problem coming from the way data was collected, we decided not only to check this particular movie (with release year being, as expected, 2010), but also other very old ones. In our analysis we haven't found any other incorrect value.
4. Another problem we had, was lacking of clear choice of features needed for our analysis. Since we find this argument as a crucial one, we'll pay strong attention to the feature selection. For the sake of broadening our horizon, we decided to extend our original dataset by additional information obtained from IMDb pro (paid, extended version of IMDb).

## 1. Load the reformatted data obtained in Milestone 2 <a class="anchor" id="chapter1"></a>

In [2]:
df = pd.read_csv(PATH_IN + 'experimental_movie_metadata.csv')
df.head(3)

,Wikipedia ID,Freebase ID,Name,Release date,Runtime,Languages,Countries,Genres,IMDb ID,averageRating,numVotes
0,975900,/m/03vyhn,Ghosts of Mars,20010824.0,98.0,['English'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",tt0228333,4.9,55240
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,20000216.0,95.0,['English'],['United States of America'],"['Mystery', 'Biographical film', 'Drama', 'Cri...",tt0245916,5.9,66
2,28463795,/m/0crgdbh,Brun bitter,19880000.0,83.0,['Norwegian'],['Norway'],"['Crime Fiction', 'Drama']",tt0094806,5.7,40


## 2. Extend our dataset by performing scraping on IMDb pro <a class="anchor" id="chapter2"></a>

In order to include more details about movies in our analysis, we wrote a small scraping script that obtains the following, additional features:
* budgets used for producing a movie
* mpaas, i.e. rating provided by Motion Picture Association of America
* box_offices, total revenues obtained by a movie
* directors, list of people responsible for direction
* writers, list of people responsible for script
* producers, list of people responsible for production
* composers, list of people responsible for music
* cinematographers, list of people responsible for photographing/ recording
* editors, list of people responsible for editing.

Due to privacy reasons, we decided to keep both headers and cookies secret as they are personal details, indicating one of the users (i.e. one of our teammates).

For features that require individual approach (budgets, box offices, mpaas) we perform separate scrapings, but for people responsible for movies we use scraping_people function, since html-wise they are better-structured on IMDb pro.

In [80]:
# cookies = {cookies}
# headers = {headers}

In [92]:
"""
    function that scraps data related to movie creators, i.e. directors, producers,
    writers, composers, editors, cinematographers
"""
def scraping_people(df, role, cookies, headers):
    values = []
    for ix, id in enumerate(df['IMDb ID']):
        print(ix)
        html_text = requests.get('https://pro.imdb.com/title/' + id, cookies=cookies, headers=headers)
        soup = BeautifulSoup(html_text.content, 'html.parser')
        try:
            values_soup = []
            values_helper = soup.find('div', {'id': role + '_summary'}).find_all('a', class_ = 'a-size- a-align- a-link- ttip')
            for value in range(len(values_helper)):
                values_soup.append(values_helper[value].get_text())
        except AttributeError:
            values_soup = None
        values.append(values_soup)
    return values

In [ ]:
directors = scraping_people(df_extended, 'director', cookies, headers)
producers = scraping_people(df_extended, 'producer', cookies, headers)
writers = scraping_people(df_extended, 'writer', cookies, headers)
composers = scraping_people(df_extended, 'composer', cookies, headers)
editors = scraping_people(df_extended, 'editor', cookies, headers)
cinematographers = scraping_people(df_extended, 'cinematographer', cookies, headers)

In [23]:
budgets, mpaas, box_offices = [], [], []

for ix, id in enumerate(df['IMDb ID']):
    print(ix)
    html_text = requests.get('https://pro.imdb.com/title/' + id, cookies=cookies, headers=headers)
    soup = BeautifulSoup(html_text.content, 'html.parser')
    #budget
    try:
        length = soup.find('span', class_ = 'a-section a-spacing-small budget_summary').get_text()
        budget = int(''.join(filter(str.isdigit, budget)))
    except AttributeError:
        budget = None
    budgets.append(budget)
    # mpaa
    try:
        mpaa = soup.find('span', class_ = 'a-color-tertiary').get_text().replace(' ', '').replace('\n', '')
    except AttributeError:
        mpaa = None
    mpaas.append(mpaa)
    # box office
    try:
        box_office = soup.find('div', class_ = 'a-section a-spacing-small gross_world_summary').get_text()
        box_office = int(''.join(filter(str.isdigit, box_office)))
    except AttributeError:
        try:
            box_office = soup.find('div', class_ = 'a-section a-spacing-small gross_usa_summary').get_text()
            box_office = int(''.join(filter(str.isdigit, box_office)))
        except AttributeError:
            box_office = None
    box_offices.append(box_office)

NameError: name 'df' is not defined

In [ ]:
dict1 = {'Budgets': budgets, 'Mpaas': mpaas, 'Box offices': box_offices, 'Directors': directors, 'Writers': writers, 'Producers': producers, 'Composers': composers, 'Cinematographers': cinematographers, 'Editors': editors}
df1 = pd.DataFrame(dict1)

In [ ]:
df_extended = pd.concat([df, df1], axis = 1)

In [19]:
df_extended.head(3)

,Wikipedia ID,Freebase ID,Name,Release date,Runtime,Languages,Countries,Genres,IMDb ID,averageRating,numVotes,Budgets,Mpaas,Box offices,Directors,Writers,Producers,Composers,Cinematographers,Editors
0,975900,/m/03vyhn,Ghosts of Mars,20010824.0,98.0,['English'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",tt0228333,4.9,55240,28000000.0,R,14010832.0,['John Carpenter'],"['Larry Sulkis', 'John Carpenter']",['Sandy King'],"['Anthrax', 'John Carpenter']",['Gary B. Kibbe'],['Paul C. Warschilka']
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,20000216.0,95.0,['English'],['United States of America'],"['Mystery', 'Biographical film', 'Drama', 'Cri...",tt0245916,5.9,66,NaN,NaN,NaN,['Edward Lucas'],"['Michael A. Graham', 'Ted Haimes']",['Michael Shevloff'],['Taylor M. Uhler'],['Shawn Maurer'],['David Post']
2,28463795,/m/0crgdbh,Brun bitter,19880000.0,83.0,['Norwegian'],['Norway'],"['Crime Fiction', 'Drama']",tt0094806,5.7,40,NaN,NaN,NaN,['Sølve Skagen'],"['Sølve Skagen', 'Gunnar Staalesen']",['Dag Alveberg'],"['Geir Bøhren', 'Bent Åserud']",['Erling Thurmann-Andersen'],['Malte Wadman']


 ## 3. Preprocessing <a class="anchor" id="chapter3"></a>

In this section we perform final preprocessing steps including:
1. Removing duplicates, since we noticed that we have around 900 repetitions.
2. Fixing a typo, being a release year 1010 instead of 2010
3. Converting strings of lists into list of strings for easier handling the data
4. Filtering out movie types we don't want to keep (e.g. series)
5. Adding weighted rating column for having metric similar to the one used by IMDb for their ranking
6. Correcting runtimes
7. Cleaning countries and languages

In [ ]:
# remove duplicates
df_extended = df_extended.drop_duplicates()

In [ ]:
# fixing a typo where a movie was produced in 1010 (correct one is 2010, we checked it manually)
df_extended.loc[51782, 'Release date'] = 20101202

In [ ]:
# Convert string of a list, to a list
# eg convert '['English', 'French']' to ['English', 'French']
to_convert = ['Languages', 'Countries', 'Genres', 'Directors', 'Writers', 'Producers', 'Composers', 'Cinematographers', 'Editors']
for elem in to_convert:
    for x in range(len(df_extended)):
        try:
            df_extended[elem][x] = ast.literal_eval(df_extended_copy[elem][x])
        except ValueError:
            df_extended[elem][x] = df_extended_copy[elem][x]

We can summarize information about our dataset as follows:

In [147]:
df_extended.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66446 entries, 0 to 66445
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Wikipedia ID      66446 non-null  int64  
 1   Freebase ID       66446 non-null  object 
 2   Name              66446 non-null  object 
 3   Release date      62290 non-null  float64
 4   Runtime           54276 non-null  float64
 5   Languages         66446 non-null  object 
 6   Countries         66446 non-null  object 
 7   Genres            66446 non-null  object 
 8   IMDb ID           66446 non-null  object 
 9   averageRating     66446 non-null  float64
 10  numVotes          66446 non-null  int64  
 11  Budgets           16167 non-null  float64
 12  Mpaas             38272 non-null  object 
 13  Box offices       17633 non-null  float64
 14  Directors         65667 non-null  object 
 15  Writers           61378 non-null  object 
 16  Producers         54028 non-null  object

We can observe that for attributes box offices and budgets, most of the values are missing. However, since for each of them we have at least 16k values, we conclude that analyzing them might be already informative and can bring interesting insights.

For descriptive analysis, we start with 'describe' function:

In [148]:
df_extended.describe()

,Wikipedia ID,Release date,Runtime,averageRating,numVotes,Budgets,Box offices
count,6.644600e+04,6.229000e+04,5.427600e+04,66446.000000,6.644600e+04,1.616700e+04,1.763300e+04
mean,1.650500e+07,1.979320e+07,1.159863e+02,6.236801,1.100798e+04,3.700631e+07,2.709415e+07
std,1.095881e+07,2.706520e+05,4.633204e+03,1.152128,6.373588e+04,4.689300e+08,8.904326e+07
min,3.300000e+02,1.888000e+07,0.000000e+00,1.000000,5.000000e+00,0.000000e+00,1.000000e+00
25%,6.184091e+06,1.959013e+07,8.400000e+01,5.600000,1.110000e+02,8.000000e+05,1.780950e+05
50%,1.604821e+07,1.986120e+07,9.400000e+01,6.400000,5.010000e+02,3.900000e+06,1.820049e+06
75%,2.607127e+07,2.003093e+07,1.070000e+02,7.000000,2.311000e+03,1.600000e+07,1.440156e+07
max,3.750192e+07,2.016032e+07,1.079281e+06,10.000000,2.659398e+06,3.000000e+10,2.922918e+09


Here, we can observe several things:
1. Max runtime is clearly an outlier, but because of its appearance we decided to write another script to double-check the lengths of the movies, based on the information provided on IMDb pro.
2. Min release date is 1888 and as we manually checked, it's a valid movie.
3. There is one movie with box office 1$. Since this exception doesn't have huge influence on our analysis, we decide to keep it. Considering we have such outliers, in our exploratory data analysis we'll use interquartile range rather than e.g. standard deviation, since IQR is less sensible to outliers.

In [15]:
df_extended.head(3)

,Wikipedia ID,Freebase ID,Name,Release date,Runtime,Languages,Countries,Genres,IMDb ID,averageRating,numVotes,Budgets,Mpaas,Box offices,Directors,Writers,Producers,Composers,Cinematographers,Editors
0,975900,/m/03vyhn,Ghosts of Mars,20010824.0,98.0,['English'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",tt0228333,4.9,55240,28000000.0,R,14010832.0,['John Carpenter'],"['Larry Sulkis', 'John Carpenter']",['Sandy King'],"['Anthrax', 'John Carpenter']",['Gary B. Kibbe'],['Paul C. Warschilka']
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,20000216.0,95.0,['English'],['United States of America'],"['Mystery', 'Biographical film', 'Drama', 'Cri...",tt0245916,5.9,66,NaN,NaN,NaN,['Edward Lucas'],"['Michael A. Graham', 'Ted Haimes']",['Michael Shevloff'],['Taylor M. Uhler'],['Shawn Maurer'],['David Post']
2,28463795,/m/0crgdbh,Brun bitter,19880000.0,83.0,['Norwegian'],['Norway'],"['Crime Fiction', 'Drama']",tt0094806,5.7,40,NaN,NaN,NaN,['Sølve Skagen'],"['Sølve Skagen', 'Gunnar Staalesen']",['Dag Alveberg'],"['Geir Bøhren', 'Bent Åserud']",['Erling Thurmann-Andersen'],['Malte Wadman']


Now we want to filter out all non-movie titles, as we don't want to include any series in our analysis. To do so, we'll use IMDb dataset again, but with primarily focus on so-called "title types".

In [16]:
title_basics = pd.read_csv(PATH_IN + 'title_basics.tsv', sep='\t')

/var/folders/22/z208ks5x5vv4dfqj0fjxt9l00000gn/T/ipykernel_54719/2459562363.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics = pd.read_csv(PATH_IN + 'title_basics.tsv', sep='\t')


In [17]:
title_basics['titleType'].unique()

array(['short', 'movie', 'tvSeries', 'tvShort', 'tvMovie', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

We'll remove the following types from our dataset: tvSeries, tvShort, tvEpisode, tvMiniSeries, videoGame, tvPilot

In [18]:
def remove_rows(df, col, values):
    return df[~df[col].isin(values)]

In [19]:
# keeping only desirable types
title_basics = remove_rows(title_basics, 'titleType', ['tvSeries', 'tvEpisode', 'tvMiniSeries', 'videoGame', 'tvPilot'])

In [20]:
# filtering our dataset
df_extended = df_extended[df_extended['IMDb ID'].isin(title_basics['tconst'])]

In [21]:
df_extended.reset_index(inplace = True)
df_extended = df_extended.drop(columns=['index'])

Following the suggestions related to weighted ratings (e.g. here https://stats.stackexchange.com/questions/6418/rating-system-taking-account-of-number-of-votes) we decide to add a column with ratings weighted by number of votes given to a movie.
The formula is (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C, where
* R = average for the movie (mean) = (Rating)
* v = number of votes for the movie = (votes)
* m = minimum votes required to be listed in the Top 250 (currently 25000)
* C = the mean vote across the whole dataset.

Applying this formula causes significant decrease in ratings for movies with a low number of votes, setting it close to the mean of averageRating column. Since we want to have a closer look on movies with many high ratings, to avoid the situation that a movie with 4 ratings being equal 8/10 is 'better' than a movie with 40000 ratings being equal 7.9/10 on average. Like IMDb, we believe that films rated by a small number of people, should not have a key influence on the selection of the most successful movie in the database.

In [24]:
# Weighted rating
mean_movie_rating = df_extended['averageRating'].mean()
print("Average rating of all movies: ", mean_movie_rating)

def calculate_wr(movie):
    return (movie['numVotes'] / (movie['numVotes'] + 25000)) * movie['averageRating'] + (25000/(movie['numVotes'] + 25000)) * mean_movie_rating

df_extended["Weighted Rating"] = df_extended.apply(lambda movie : calculate_wr(movie), axis = 1)

Average rating of all movies:  6.229809548418477


In [25]:
df_extended_copy = df_extended.copy()

In [26]:
# Convert string of a list, to a list of strings in order to treat content of our data as separate, meaningful elements
# e.g. convert '['English', 'French']' to ['English', 'French']
to_convert = ['Languages', 'Countries', 'Genres', 'Directors', 'Writers', 'Producers', 'Composers', 'Cinematographers', 'Editors']
for elem in to_convert:
    for x in range(len(df_extended)):
        try:
            df_extended[elem][x] = ast.literal_eval(df_extended_copy[elem][x])
        except ValueError:
            df_extended[elem][x] = df_extended_copy[elem][x]

/var/folders/22/z208ks5x5vv4dfqj0fjxt9l00000gn/T/ipykernel_54719/3862521430.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extended[elem][x] = ast.literal_eval(df_extended_copy[elem][x])
/var/folders/22/z208ks5x5vv4dfqj0fjxt9l00000gn/T/ipykernel_54719/3862521430.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extended[elem][x] = df_extended_copy[elem][x]


Next, we want to replace all incorrect runtimes in our dataset by lengths of the movies taken from IMDb. As we have seen, current longest movie lasts more than 1 million minutes. To tackle this problem, we run another scraping script.

In [ ]:
lengths = []

for ix, id in enumerate(df_extended['IMDb ID']):
    print(ix)
    html_text = requests.get('https://pro.imdb.com/title/' + id, cookies=cookies, headers=headers)
    soup = BeautifulSoup(html_text.content, 'html.parser')
    # runtimes
    try:
        length = int(np.floor(float(soup.find('span', {'id': 'running_time'}).get_text().replace(' ', '').replace('\n', '').replace('min', '').replace("'", '').replace(',', ''))))
    except AttributeError:
        length = None
    lengths.append(length)

all_lengths = pd.DataFrame(lengths)

In [33]:
df_extended['Lengths'] = all_lenghts['0']

In [34]:
# function that takes as input two dataframe columns list1 and list2 and if there is a value in list1, it is put as a value in list2
def replace_values_in_list2(df, imdb, runtime):
    df[runtime] = df.apply(lambda x: x[imdb] if np.isnan(x[imdb]) == False else x[runtime], axis = 1)
    return df

In [35]:
df_extended = replace_values_in_list2(df_extended, 'Lengths', 'Runtime')
df_extended = df_extended.drop(['Lengths'], axis = 1)

During careful manual investigation we discovered many typos and inconsistencies in both 'Countries' and 'Languages' columns. Thus, for those two attributes we create dictionaries in order to fix and unify invalid values. Additionally, we group countries that appear under different, historical names, but are related to the same country.

Since in our dataset we have the United Kingdom, we decided to include all available parts of the UK (Scotland, Ireland, etc.) under one common name United Kingdom. We believe that this approach might be helpful for spatial analysis. If we didn't group those countries we would have obtained many lists with few movies for each. Lastly, this unification is natural, since all of them belong to the UK.

Correcting languages properly and fully would require both enormous amount of time and expert knowledge. Comparing to countries, languages and dialects are much more diverse, and as such are less known. Thus, we correct only the ones that are reasonably easy to detect, keeping in mind that cleaning them completely could be a useful improvement for the future analysis.

In [37]:
def unique_values_in_column(df, column_name):
    all_values = list(df[column_name])
    unique_values = list(set([item for sublist in all_values for item in sublist]))
    return unique_values

In [38]:
# function that for each lists in a dataframe column replaces values from another list with a new value
def replace_values_in_list(df, col, old_values, new_value):
    df[col] = df[col].apply(lambda x: [new_value if check_common_elements(x, old_values) else i for i in x])
    return df

# function that checks if two lists have elements in common
def check_common_elements(list1, list2):
    return bool(set(list1) & set(list2))

In [39]:
countries_to_change = {}
countries_to_change['Germany'] = ['West Germany', 'German Language',   'Weimar Republic',  'Germany',  'Nazi Germany',  'German Democratic Republic']
countries_to_change['United Kingdom'] = ['Kingdom of Great Britain', 'United Kingdom',  'Northern Ireland',  'England',  'Wales',  'Scotland',  'Isle of Man']
countries_to_change['Italy'] = ['Kingdom of Italy', 'Italy']
countries_to_change['India'] = ['Malayalam Language',  'India']
countries_to_change['Palestine'] = ['Palestinian Territories',  'Mandatory Palestine', 'Palestinian territories']
countries_to_change['Congo'] = ['Democratic Republic of the Congo',  'Congo']
countries_to_change['Iraq'] = ['Iraqi Kurdistan', 'Iraq']
countries_to_change['Uzbekistan'] = ['Uzbek SSR']
countries_to_change['Yugoslavia'] = ['Yugoslavia',  'Federal Republic of Yugoslavia',  'Socialist Federal Republic of Yugoslavia']
countries_to_change['Georgia'] = ['Georgia', 'Georgian SSR']
countries_to_change['China'] = ['Macau',  'China',  'Republic of China']
countries_to_change['Ukraine'] = ['Ukrainian SSR',  'Ukraine',  'Ukranian SSR']
countries_to_change['Russia'] = ['Crime',  'Soviet occupation zone',  'Soviet Union', 'Russia']
countries_to_change['Slovakia'] = ['Slovak Republic',  'Slovakia', 'Slovak Republic']
countries_to_change['Korea'] = ['South Korea', 'Korea']
countries_to_change['The Netherlands'] = ['Aruba',  'Netherlands']

In [40]:
for country in countries_to_change:
    df_extended = replace_values_in_list(df_extended, 'Countries', countries_to_change[country], country)

In [41]:
languages_to_change = {}
languages_to_change['Egyptian'] = ['Egyptian,']
languages_to_change['Apache'] = ['Apache,']
languages_to_change['Thai'] = ['Thai,']
languages_to_change['French'] = ['France']
languages_to_change['English'] = ['American', 'English']
languages_to_change['Chinese'] = ['Mandarin', 'Chinese', 'Chinese,', 'Cantonese']
languages_to_change['German'] = ['Deutsch', 'German']
languages_to_change['Frisian'] = ['Frisian,']
languages_to_change['Saami'] = ['Saami,']
languages_to_change['Maya'] = ['Maya,']
languages_to_change['Khmer'] = ['Khmer,']
languages_to_change['Fulfulde'] = ['Fulfulde,']

In [42]:
for language in languages_to_change:
    df_extended = replace_values_in_list(df_extended, 'Languages', languages_to_change[language], language)

After the replacements, we have repetitions in some lists (e.g. ['American', 'English'] -> ['English', 'English']. To fix them, we change those lists into sets and then back to lists.

In [ ]:
df_extended['Languages'] = df_extended['Languages'].apply(lambda x: list(set(x)))
df_extended['Countries'] = df_extended['Countries'].apply(lambda x: list(set(x)))

In [51]:
df_extended.to_csv(PATH_IN + 'final_dataset.csv', sep = ',', index = False)

In [95]:
df_extended.head(3)

,Wikipedia ID,Freebase ID,Name,Release date,Runtime,Languages,Countries,Genres,IMDb ID,averageRating,...,Budgets,Mpaas,Box offices,Directors,Writers,Producers,Composers,Cinematographers,Editors,Weighted Rating
0,975900,/m/03vyhn,Ghosts of Mars,20010824.0,98.0,[English],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",tt0228333,4.9,...,28000000.0,R,14010832.0,[John Carpenter],"[Larry Sulkis, John Carpenter]",[Sandy King],"[Anthrax, John Carpenter]",[Gary B. Kibbe],[Paul C. Warschilka],5.314323
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,20000216.0,95.0,[English],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",tt0245916,5.9,...,NaN,NaN,NaN,[Edward Lucas],"[Michael A. Graham, Ted Haimes]",[Michael Shevloff],[Taylor M. Uhler],[Shawn Maurer],[David Post],6.228941
2,28463795,/m/0crgdbh,Brun bitter,19880000.0,83.0,[Norwegian],[Norway],"[Crime Fiction, Drama]",tt0094806,5.7,...,NaN,NaN,NaN,[Sølve Skagen],"[Sølve Skagen, Gunnar Staalesen]",[Dag Alveberg],"[Geir Bøhren, Bent Åserud]",[Erling Thurmann-Andersen],[Malte Wadman],6.228963
